### 체인을 활용한 고급 기법 적용

<img src="../cover.png" 
     alt="NLP와 LLM 실전 가이드(한빛미디어)"
     style="border: 3px solid gray; box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.3); border-radius: 10px; width: 300px;" width="300">


* 저자:  
    - [Lior Gazit](https://www.linkedin.com/in/liorgazit).  
    - [Meysam Ghaffari](https://www.linkedin.com/in/meysam-ghaffari-ph-d-a2553088/).
* 역자:
    - [박조은](https://github.com/corazzon)
* 이 노트북은 다음의 책에서 소개하는 내용입니다.
    - 역서 : NLP와 LLM 실전 가이드(한빛미디어)
    - 원서 : [Mastering NLP from Foundations to LLMs](https://www.amazon.com/dp/1804619183)

colab 실습 : 
https://github.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/Mastering-NLP-from-Foundations-to-LLMs/blob/main/Chapter9_notebooks/Ch9_Advanced_LangChain_Configurations_and_Pipeline.ipynb)  


원서 Colab 실습 :   
https://github.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs   
<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter9_notebooks/Ch9_Advanced_Methods_with_Chains.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**노트북의 목적:**  
**Langchain**을 활용하여 다음과 같은 방법들을 탐구합니다:  
* LLM을 통한 일반 지식 질문하기  
* LLM 응답으로부터 구조화된 데이터 형식 도출하기  
* 대화 내에서 메모리 기능 설정하기  

**필요 조건:**  
* Colab에서 실행 시 다음 런타임 노트북 설정이 필요: `Python3, CPU`  
* 이 코드는 LLM으로 OpenAI의 API를 사용하므로 유료 **API 키**가 필요합니다.

>*```면책사항: 이 노트북에서 다루는 내용과 아이디어는 저자들 개인의 것이며, 저자들의 고용주의 견해나 지적 재산을 대변하지 않습니다.```*

설치:

In [1]:
# 참고사항:
# 아래 코드가 Python 패키지 의존성 문제로 오류가 발생하면, 이는 새로운 버전의 영향일 수 있습니다.
# 이럴 때 default_installations를 False로 설정하여 원저자가 설정한 의존성을 따르게 할 수 있습니다.
default_installations = True
if default_installations:
    !pip install -qU langchain openai langchain-openai
    !pip install -qU langchain-community
else:
    import requests
    text_file_path = "requirements__Ch9_Advanced_Methods_with_Chains.txt"
    url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter9_notebooks/" + text_file_path
    res = requests.get(url)
    with open(text_file_path, "w") as f:
        f.write(res.text)

    !pip install -r requirements__Ch9_Advanced_Methods_with_Chains.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


Imports:

In [2]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import CommaSeparatedListOutputParser
import os
import pandas as pd
import json

코드 설정:

OpenAI API key:  
**문자열 형태로 아래 "..."에 OpenAI에서 발급받은 key를 입력해 주세요!**  


Colab 보안 비밀 설정은 왼쪽 열쇠 모양의 아이콘을 클릭하면 나옵니다.
<img src="https://i.imgur.com/7P383n4.png" width="500">

유료 LLM이 아닌 무료 LLM을 활용하고자 한다면, 책에서 설명하는 허깅페이스를 활용하는 예제를 따라 대체 방법을 사용해 보세요.

In [3]:
os.environ["OPENAI_API_KEY"] = "..."

# Colab 에서는 보안 비밀키 설정을 통해 API 키를 매번 입력하지 않고 아랴와 같이 관리할 수 있습니다.
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    if not os.environ["OPENAI_API_KEY"]:
        raise ValueError("Colab의 보안 비밀 설정에서 'OPENAI_API_KEY'를 찾을 수 없습니다.")
except ImportError:
    print("Colab 환경이 아니므로, 'api_key' 변수에 직접 값을 할당해야 합니다.")

In [4]:
# llm = OpenAI(model_name='gpt-4o-mini')

### LLM에 일반적인 지식 질문하기

In [5]:
simple_question = "메탈리카(Metallica) 멤버는 누구인가요? 쉼표로 구분해 나열해 주세요. 멤버 이름 외 다른 텍스트는 생성하지 마세요."

chat = ChatOpenAI(model_name="gpt-4o-mini")  # 또는 사용 가능한 다른 ChatGPT 모델
chat_prompt = ChatPromptTemplate.from_template(simple_question)
llm_chain = LLMChain(llm=chat, prompt=chat_prompt)

print(llm_chain.run({}))

<ipython-input-5-1f31b9718770>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model_name="gpt-4o-mini")  # 또는 사용 가능한 다른 ChatGPT 모델
<ipython-input-5-1f31b9718770>:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=chat, prompt=chat_prompt)
<ipython-input-5-1f31b9718770>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run({}))


제임스 헷필드, 라스 울리히, 키스 햄릿, 로버트 트루히요


### LLM이 특정 데이터 형식으로 출력을 제공하도록 설정하기

In [6]:
request_list_format = """주기율표에서 처음 10개의 원소를 쉼표로 구분하여 작성해 주세요.
원소 외에 다른 텍스트는 생성하지 마세요."""

output_parser = CommaSeparatedListOutputParser()
conversation = LLMChain(
    llm = ChatOpenAI(  # ChatGPT 모델을 사용할 경우 ChatOpenAI로 변경
        model="gpt-4o-mini",  # 또는 "gpt-4o"
        temperature=0.1,
    ),
    output_parser=output_parser,
    prompt=PromptTemplate(template=request_list_format, input_variables=[])
)

print(conversation.predict())

['수소', '헬륨', '리튬', '베릴륨', '붕소', '탄소', '질소', '산소', '플루오르', '네온']


### 자연스러운 대화를 위해 발전하기
* 이전 상호작용을 참조와 맥락으로 활용할 수 있도록 메모리 요소를 추가해 후속 프롬프트를 이어가는 방식

In [7]:
# Chat LLM 구성
llm = ChatOpenAI(
    model="gpt-4o-mini",  # 또는 "gpt-4"
    temperature=0.3,
)

# 대화 프롬프트 설정
request_for_continuous_conversation = """
현재 대화 내용:
{history}

다음 질문에 답변해주세요:
{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=request_for_continuous_conversation,
                          input_variables=["history", "input"]),
    memory=ConversationBufferMemory()
)

# 대화 예측 실행
result = conversation.predict_and_parse(input="알고 있는 공휴일 10가지를 쉼표로 구분된 목록으로 작성해 주세요.")
print(result)


<ipython-input-7-e36fdf3dcbba>:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
<ipython-input-7-e36fdf3dcbba>:15: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:341: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


다음은 알고 있는 공휴일 10가지입니다: 설날, 추석, 어린이날, 성탄절, 광복절, 개천절, 노동절, 한글날, 크리스마스, 발렌타인데이.


In [8]:
conversation.predict_and_parse(input="방금 나열한 공휴일 목록에서 종교적이지 않은 공휴일을 제거해 주세요. ")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:341: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'종교적이지 않은 공휴일을 제거한 목록은 다음과 같습니다: 설날, 추석, 어린이날, 광복절, 개천절, 노동절, 한글날.'

In [9]:
advanced_data_structure = """각 공휴일에 대해 두 문장으로 설명하세요. 출력은 JSON 형식의 표로 작성해 주세요.
표의 이름은 "holidays"이고, 필드는 "name"과 "description"입니다. 각 행에서 "name"은 공휴일의 이름이고,
"description"은 생성된 설명입니다. 출력의 구문은 줄 바꿈 없이 JSON 형식이어야 합니다.

예시:
{"holidays": [
        {"name": "holiday_name",
         "description": "holiday_description"
        }
        ]}
"""

output = conversation.predict(input=advanced_data_structure)
print(output)


{"holidays": [
        {"name": "설날", "description": "설날은 한국의 전통 명절로, 음력 1월 1일에 해당합니다. 가족들이 모여 차례를 지내고 떡국을 먹으며 새해를 맞이합니다."},
        {"name": "추석", "description": "추석은 한국의 가을 명절로, 음력 8월 15일에 해당합니다. 이 날은 조상에게 감사의 마음을 전하고 송편을 만들어 가족과 함께 나누는 날입니다."},
        {"name": "어린이날", "description": "어린이날은 매년 5월 5일에 기념되는 날로, 어린이의 권리와 행복을 기념합니다. 이 날은 부모들이 어린이들에게 선물과 특별한 시간을 선사하는 날입니다."},
        {"name": "광복절", "description": "광복절은 1945년 8월 15일, 한국이 일본의 식민지에서 해방된 날을 기념합니다. 이 날은 국가의 독립과 자유를 기념하며 다양한 행사와 기념식이 열립니다."},
        {"name": "개천절", "description": "개천절은 10월 3일에 기념되는 날로, 한국의 건국을 기념합니다. 이 날은 단군이 고조선을 세운 날로 여겨지며, 다양한 행사와 퍼레이드가 열립니다."},
        {"name": "노동절", "description": "노동절은 매년 5월 1일에 기념되는 날로, 노동자의 권리와 노사 간의 연대를 강조합니다. 이 날은 전 세계적으로 노동자들의 기념일로 다양한 행사와 집회가 열립니다."},
        {"name": "한글날", "description": "한글날은 10월 9일에 기념되는 날로, 한글의 창제를 기념합니다. 이 날은 한글의 중요성을 되새기고, 한국어와 한글의 발전을 기념하는 행사들이 진행됩니다."}
]}


In [10]:
pd.set_option('display.max_colwidth', None)

dict = json.loads(output)
pd.json_normalize(dict["holidays"]).style.set_properties(**{'text-align': 'left'})

,name,description
0,설날,"설날은 한국의 전통 명절로, 음력 1월 1일에 해당합니다. 가족들이 모여 차례를 지내고 떡국을 먹으며 새해를 맞이합니다."
1,추석,"추석은 한국의 가을 명절로, 음력 8월 15일에 해당합니다. 이 날은 조상에게 감사의 마음을 전하고 송편을 만들어 가족과 함께 나누는 날입니다."
2,어린이날,"어린이날은 매년 5월 5일에 기념되는 날로, 어린이의 권리와 행복을 기념합니다. 이 날은 부모들이 어린이들에게 선물과 특별한 시간을 선사하는 날입니다."
3,광복절,"광복절은 1945년 8월 15일, 한국이 일본의 식민지에서 해방된 날을 기념합니다. 이 날은 국가의 독립과 자유를 기념하며 다양한 행사와 기념식이 열립니다."
4,개천절,"개천절은 10월 3일에 기념되는 날로, 한국의 건국을 기념합니다. 이 날은 단군이 고조선을 세운 날로 여겨지며, 다양한 행사와 퍼레이드가 열립니다."
5,노동절,"노동절은 매년 5월 1일에 기념되는 날로, 노동자의 권리와 노사 간의 연대를 강조합니다. 이 날은 전 세계적으로 노동자들의 기념일로 다양한 행사와 집회가 열립니다."
6,한글날,"한글날은 10월 9일에 기념되는 날로, 한글의 창제를 기념합니다. 이 날은 한글의 중요성을 되새기고, 한국어와 한글의 발전을 기념하는 행사들이 진행됩니다."
